In [2]:
import sys
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

from jax import config
config.update("jax_enable_x64", True)

In [3]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate a random SDP.
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 2.6543470585557296
A solution X is
[[ 1.60805504 -0.59770125 -0.69575821]
 [-0.59770125  0.22228555  0.24689067]
 [-0.69575821  0.24689067  1.39679134]]


In [4]:
from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, factorize_psd, super2choi
d, N = 2, 4
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=1e-2)
tau = 4
superops_exp = []
for i, op in enumerate([Lvec, Lvec_odd, Lvec_even, Lnn]):
    if i == 1 or i == 3:
        superops_exp.append(exp_operator_dt(op, tau/2, 'jax'))
    else:
        superops_exp.append(exp_operator_dt(op, tau, 'jax'))
exp_Lvec, exp_Lvec_odd, exp_Lvec_even, exp_Lnn = superops_exp

X1 = factorize_psd(psd=super2choi(exp_Lvec_odd))
X2 = factorize_psd(psd=super2choi(exp_Lvec_even))

from opentn.optimization import gds, frobenius_norm, model_Ys, compute_loss
cost_n4, grads_n4, xi_n4 = gds(fn=compute_loss, x0=[X1, X2, X1], iter=25, loss_fn=frobenius_norm, model=model_Ys, rate=1.5e-9, exact = exp_Lvec, show_cost=False)
cost_n4[-1]

DeviceArray(2.15352749e-07, dtype=float64)

In [5]:
# now that we have the other two, lets use it for an optimization of Y1
from opentn.optimization import model_Zs

X2 = xi_n4[-1][1]
X3 = xi_n4[-1][2]
Y2 = X2@X2.conj().T
Y3 = X3@X3.conj().T

In [14]:
type(X2)

jaxlib.xla_extension.DeviceArray

# Preliminaries for CVXPY test:
- implement choi matrix composition in two different ways and see if they are equivalent

In [6]:
# lets test the two methods for choi composition
from opentn.transformations import link_product, choi_composition

# lets do the link product of Y2 and Y3
dim = d**N
Y_23 = link_product(C1=Y2, C2=Y3, dim=dim)
# now lets the same contractions with tensordot
Y_23_tensor = choi_composition(C1=Y2, C2=Y3, dim=dim)
np.allclose(Y_23, Y_23_tensor)

True

since this worked, now the plan is to take advantage of the built in functions from CVXPY that can handle the operations in link_product which (sadly) cannot be done with choi_composition since CVXPY does not support >2D arrays

In [20]:
from opentn.transformations import super2choi
# firs thing to do would to gather SDP and min square syntax:
rhs = np.asarray(super2choi(superop=exp_Lvec, dim=dim))
print(rhs.shape)
A = Y3@Y2
print(rhs.shape)
I = np.eye(dim)
print(I.shape)
IxA = np.kron(I, A)
print(IxA.shape)


(256, 256)
(256, 256)
(16, 16)
(4096, 4096)


In [24]:
from scipy import sparse
srhs = sparse.csr_matrix(rhs)
sIxA = sparse.csr_matrix(IxA)
sI = sparse.csr_matrix(I)

In [25]:
# The operator >> denotes matrix inequality.
X = cp.Variable((dim**2,dim**2), PSD=True)
lhs =  cp.partial_trace(cp.kron(sI, cp.partial_transpose(X, dims=[dim, dim], axis=0)) @ sIxA, dims=[dim, dim, dim], axis=1)
cost = cp.sum_squares(lhs - rhs)
prob = cp.Problem(cp.Minimize(cost))

/Users/emiliano_gr/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/cvxpy/problems/problem.py:155: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


In [26]:
prob.solve()

/Users/emiliano_gr/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/cvxpy/problems/problem.py:161: UserWarning: Constraint #0 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/Users/emiliano_gr/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/cvxpy/problems/problem.py:161: UserWarning: Constraint #1 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
